<a href="https://colab.research.google.com/github/yuu067/MIA-IABD-2425/blob/main/UD03/notebooks/EX2.-classificador_preguntes_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios UD03_02

## Clasificar preguntas

En la práctica [Clasificación de texto con PyTorch](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/2.-classificacio_text_torch_ES.ipynb) hemos visto el proceso para convertir un texto en una representación numérica que pueda ser utilizada por un algoritmo de aprendizaje automático. Hemos visto diferentes representaciones como *Bolsa de palabras* (BoW) y *incrustaciones de palabras* (word embeddings) y cómo entrenar una red neuronal para clasificar texto.

En esta práctica, deberá repetir el proceso para clasificar las preguntas en temas. Usaremos el conjunto de datos `Trec` que contiene preguntas en inglés y su tema. El conjunto de datos está disponible en [trec](https://huggingface.co/datasets/CogComp/trec).

### Objetivos de la práctica
* Reproducir el proceso visto en la práctica [Clasificación de texto con PyTorch](https://colab.research.google.com/github/martinezpenya/MIA-IABD-2425/blob/main/UD03/notebooks/2.-classificacio_text_torch_ES.ipynb) para clasificar preguntas en temáticas.
* Deberá preparar una red neuronal con PyTorch para clasificar las preguntas.
* Pruebe las diferentes representaciones vistas para convertir el texto en una representación numérica.
* Tendrá que comparar los resultados obtenidos con las diferentes representaciones.

In [1]:
%pip install --upgrade torch datasets scikit-learn transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset

dataset = load_dataset('trec')

dataset

The repository for trec contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/trec.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 500
    })
})

In [6]:
print(dataset['train'][0])

print(dataset['train'].features)

classes = dataset['train'].features["coarse_label"].names
classes

{'text': 'How did serfdom develop in and then leave Russia ?', 'coarse_label': 2, 'fine_label': 26}
{'text': Value(dtype='string', id=None), 'coarse_label': ClassLabel(names=['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM'], id=None), 'fine_label': ClassLabel(names=['ABBR:abb', 'ABBR:exp', 'ENTY:animal', 'ENTY:body', 'ENTY:color', 'ENTY:cremat', 'ENTY:currency', 'ENTY:dismed', 'ENTY:event', 'ENTY:food', 'ENTY:instru', 'ENTY:lang', 'ENTY:letter', 'ENTY:other', 'ENTY:plant', 'ENTY:product', 'ENTY:religion', 'ENTY:sport', 'ENTY:substance', 'ENTY:symbol', 'ENTY:techmeth', 'ENTY:termeq', 'ENTY:veh', 'ENTY:word', 'DESC:def', 'DESC:desc', 'DESC:manner', 'DESC:reason', 'HUM:gr', 'HUM:ind', 'HUM:title', 'HUM:desc', 'LOC:city', 'LOC:country', 'LOC:mount', 'LOC:other', 'LOC:state', 'NUM:code', 'NUM:count', 'NUM:date', 'NUM:dist', 'NUM:money', 'NUM:ord', 'NUM:other', 'NUM:period', 'NUM:perc', 'NUM:speed', 'NUM:temp', 'NUM:volsize', 'NUM:weight'], id=None)}


['ABBR', 'ENTY', 'DESC', 'HUM', 'LOC', 'NUM']

In [7]:
# Separar el conjunto de datos en entrenamiento y test
ds_train = dataset['train']
ds_test = dataset['test']
# Veamos cuántos ejemplos hay en cada set
print('Número de ejemplos de train:', len(ds_train))
print('Número de ejemplos de test:', len(ds_test))

Número de ejemplos de train: 5452
Número de ejemplos de test: 500


In [10]:
# Imprimimos los primeros 5 ejemplos del conjunto de entrenamiento
for w in ds_train.take(5):
    print(f"{w['coarse_label']} ({classes[w['coarse_label']]}) -> {w['text']}")

2 (DESC) -> How did serfdom develop in and then leave Russia ?
1 (ENTY) -> What films featured the character Popeye Doyle ?
2 (DESC) -> How can I find a list of celebrities ' real names ?
1 (ENTY) -> What fowl grabs the spotlight after the Chinese Year of the Monkey ?
0 (ABBR) -> What is the full form of .com ?


In [13]:
import torch
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
vocab = tokenizer.get_vocab()

len_vocab = len(vocab)

def encode(text):
    tk = tokenizer.tokenize(text)
    return tokenizer.convert_tokens_to_ids(tk)

def to_bow(text, tamany_vocabulari=len_vocab):
    res = torch.zeros(tamany_vocabulari, dtype=torch.float32)

    for i in encode(text):
        if i<tamany_vocabulari:
            res[i] += 1
    return res

print(ds_train[0])
print(to_bow(ds_train[0]["text"]))

{'text': 'How did serfdom develop in and then leave Russia ?', 'coarse_label': 2, 'fine_label': 26}
tensor([0., 0., 0.,  ..., 0., 0., 0.])


In [14]:
from torch.utils.data import DataLoader

def bowify(batch):
    etiquetas = torch.LongTensor([pregunta["coarse_label"] for pregunta in batch])
    preguntas = torch.stack([to_bow(pregunta["text"]) for pregunta in batch])
    return (
            etiquetas,
            preguntas
    )


train_loader = DataLoader(ds_train, batch_size=16, collate_fn=bowify)
test_loader = DataLoader(ds_test, batch_size=16, collate_fn=bowify)

In [15]:
net = torch.nn.Sequential(
    torch.nn.Linear(len(vocab), 4),
    torch.nn.LogSoftmax(dim=1)
)

In [16]:
def train_epoch(
    net,
    dataloader,
    lr=0.01,
    optimizer=None,
    loss_fn=torch.nn.NLLLoss(),
    epoch_size=None,
    report_freq=200,
):
    # Si no se especifica un optimizador, usamos Adam
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr=lr)

    # Ponemos la red en modo de entrenamiento.Esto activa el comportamiento de las capas de DropOut, por ejemplo.
    net.train()

    # Inicializar las variables que nos servirán para calcular la precisión
    total_loss, acc, count, i = 0, 0, 0, 0

    # Iteremamos sobre el dataloader
    for labels, features in dataloader:

        # Ponemos los gradientes a cero
        optimizer.zero_grad()

        # calculamos la salida de la red
        out = net(features)

        # Calculamos la pérdida. Esta función ya se aplica a Softmax a la salida.
        loss = loss_fn(out, labels)  # cross_entropy(out,labels)

        # Propagamos la pérdida de regreso. Esto hará que se calculen los gradientes .
        loss.backward()

        # Actualizamos los pesos de la red. Esto toma un paso de optimización.
        optimizer.step()

        # Actualizamos variables para calcular la precisión.
        total_loss += loss

        # Calculamos la precisión. Para hacer esto, debemos convertir la salida de red en etiquetas.
        # La clase con la mayor probabilidad es la que predecimos como etiqueta.
        _, predicted = torch.max(out, 1)
        acc += (predicted == labels).sum()

        # Actualizamos el contador de muestras
        count += len(labels)

        # Mostramos la precisión cada report_freq muestras
        i += 1
        if i % report_freq == 0:
            print(f"{count}: acc={acc.item()/count}")

        # Si se especifica epoch_size y ya hemos procesado este número de muestras, dejamos el bucle.
        if epoch_size and count > epoch_size:
            break
    return total_loss.item() / count, acc.item() / count


train_epoch(net, train_loader, epoch_size=15000)

IndexError: Target 5 is out of bounds.